In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


# 베이스 경로 설정
BASE_DIR = '/content/drive/MyDrive/Dacon_FakeText/'

Mounted at /content/drive


In [2]:
import os
import gc
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from tqdm import tqdm
from collections import defaultdict


In [3]:
# ✅ 문단별 데이터 로드
train_df = pd.read_csv(BASE_DIR + 'data/train_paragraph_with_features_v2.csv')
test_df  = pd.read_csv(BASE_DIR + 'data/test_paragraph_with_features_v2.csv')

# ✅ SBERT 임베딩 로드
X_sbert_train = np.load(BASE_DIR + 'data/embeddings/train_sbert_embeddings.npy')
X_sbert_test  = np.load(BASE_DIR + 'data/embeddings/test_sbert_embeddings.npy')



In [4]:
def get_title_agg_sbert_features_gpu_optimized(sbert_array, paragraph_df):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    sbert_tensor = torch.tensor(sbert_array, dtype=torch.float32)  # CPU 상태 유지
    num_paragraphs, dim = sbert_tensor.shape

    avg_sbert = torch.zeros_like(sbert_tensor).cpu()
    max_sbert = torch.zeros_like(sbert_tensor).cpu()
    cos_mean = torch.zeros((num_paragraphs, 1), device=device)
    cos_max = torch.zeros((num_paragraphs, 1), device=device)

    # 🔹 타이틀별 인덱스 저장
    title_groups = paragraph_df.groupby('title').indices

    # 🔹 문단 개수 기준으로 묶기 → 길이 기반 batching
    length_buckets = defaultdict(list)
    for title, idx_list in title_groups.items():
        length_buckets[len(idx_list)].append((title, idx_list))


    total_titles = sum(len(group) for group in length_buckets.values())
    with tqdm(total=total_titles, desc="Computing SBERT group features") as pbar:
        for length, group in length_buckets.items():
            for title, idx_list in group:
                idx_tensor = torch.tensor(idx_list, dtype=torch.long)
                sbert_group = sbert_tensor[idx_tensor].to(device)

                # 평균/최대
                avg_vec = sbert_group.mean(dim=0)
                max_vec = sbert_group.max(dim=0).values
                avg_sbert[idx_tensor.cpu()] = avg_vec.cpu()
                max_sbert[idx_tensor.cpu()] = max_vec.cpu()

                if length == 1:
                    cos_mean[idx_tensor, 0] = 0.0
                    cos_max[idx_tensor, 0] = 0.0
                else:
                    norm = torch.norm(sbert_group, dim=1, keepdim=True) + 1e-8
                    sbert_normalized = sbert_group / norm
                    cos_sim_matrix = sbert_normalized @ sbert_normalized.T
                    mask = torch.eye(length, device=device).bool()
                    cos_sim_matrix.masked_fill_(mask, float('nan'))

                    cos_mean_vals = torch.nanmean(cos_sim_matrix, dim=1)
                    cos_max_vals = torch.where(
                        torch.isnan(cos_sim_matrix),
                        torch.tensor(float('-inf'), device=device),
                        cos_sim_matrix
                    ).max(dim=1).values

                    cos_mean[idx_tensor, 0] = cos_mean_vals
                    cos_max[idx_tensor, 0] = cos_max_vals

                pbar.update(1)  # 현황 바 업데이트


    return (
        avg_sbert.cpu().numpy(),
        max_sbert.cpu().numpy(),
        cos_mean.cpu().numpy(),
        cos_max.cpu().numpy()
    )

In [5]:
# ✅ 관계 피처 계산 (avg / max 만 사용)
avg_train, max_train, *_ = get_title_agg_sbert_features_gpu_optimized(X_sbert_train, train_df)
avg_test, max_test, *_ = get_title_agg_sbert_features_gpu_optimized(X_sbert_test, test_df)

# ✅ SBERT avg/max 컬럼 이름 정의
avg_col_names = [f'sbert_avg_{i}' for i in range(avg_train.shape[1])]
max_col_names = [f'sbert_max_{i}' for i in range(max_train.shape[1])]

# ✅ DataFrame으로 변환
avg_train_df = pd.DataFrame(avg_train, columns=avg_col_names)
max_train_df = pd.DataFrame(max_train, columns=max_col_names)

avg_test_df = pd.DataFrame(avg_test, columns=avg_col_names)
max_test_df = pd.DataFrame(max_test, columns=max_col_names)

# ✅ 원본 df에 concat
train_df = pd.concat([train_df.reset_index(drop=True), avg_train_df, max_train_df], axis=1)
test_df  = pd.concat([test_df.reset_index(drop=True),  avg_test_df, max_test_df], axis=1)

# ✅ GPU 메모리 정리
import gc
del avg_train, max_train, avg_train_df, max_train_df, avg_test_df, max_test_df
gc.collect()
torch.cuda.empty_cache()

Computing SBERT group features: 100%|██████████| 253/253 [00:00<00:00, 4046.26it/s]


In [6]:
base_cols = [
    'char_count', 'word_count', 'sentence_count',
    'absolute_position', 'relative_position',
    'tfidf_mean', 'tfidf_max'  # cosine_mean, cosine_max 제거됨!
]


In [7]:
# ✅ numpy 변환
X_feat_train = train_df[base_cols].values
X_feat_test  = test_df[base_cols].values

scaler = StandardScaler()
X_feat_train = scaler.fit_transform(X_feat_train)
X_feat_test  = scaler.transform(X_feat_test)

avg_train = train_df[[col for col in train_df.columns if col.startswith('sbert_avg_')]].values
max_train = train_df[[col for col in train_df.columns if col.startswith('sbert_max_')]].values
avg_test = test_df[[col for col in test_df.columns if col.startswith('sbert_avg_')]].values
max_test = test_df[[col for col in test_df.columns if col.startswith('sbert_max_')]].values

# PCA 축소 (64차원 추천)
pca_avg = PCA(n_components=64, random_state=42)
pca_max = PCA(n_components=64, random_state=42)

avg_train_pca = pca_avg.fit_transform(avg_train)
avg_test_pca = pca_avg.transform(avg_test)

max_train_pca = pca_max.fit_transform(max_train)
max_test_pca = pca_max.transform(max_test)

# ✅ 최종 Concat: SBERT + SBERT avg/max + Features
X_train = np.concatenate([X_sbert_train, avg_train_pca, max_train_pca, X_feat_train], axis=1)
X_test  = np.concatenate([X_sbert_test, avg_test_pca, max_test_pca, X_feat_test], axis=1)

print(f"✅ 최종 X_train shape: {X_train.shape}")
print(f"✅ 최종 X_test shape:  {X_test.shape}")

# ✅ 저장
output_dir = BASE_DIR + 'data/embeddings/'
os.makedirs(output_dir, exist_ok=True)

np.save(output_dir + 'train_concat_nocosine.npy', X_train)
np.save(output_dir + 'test_concat_nocosine.npy', X_test)

print("✅ 문단 간 관계 포함 최종 concat 저장 완료!")


# ✅ 저장 경로
train_path = BASE_DIR + 'data/train_paragraph_nocosine.parquet'
test_path  = BASE_DIR + 'data/test_paragraph_nocosine.parquet'

# ✅ 저장
train_df.to_parquet(train_path, index=False)
print("✅ train_paragraph_1.parquet 저장 완료!")

test_df.to_parquet(test_path, index=False)
print("✅ test_paragraph_1.parquet 저장 완료!")

✅ 최종 X_train shape: (1226364, 903)
✅ 최종 X_test shape:  (1962, 903)
✅ 문단 간 관계 포함 최종 concat 저장 완료!
✅ train_paragraph_1.parquet 저장 완료!
✅ test_paragraph_1.parquet 저장 완료!


In [4]:
train_path = BASE_DIR + 'data/train_paragraph_1.parquet'
test_path  = BASE_DIR + 'data/test_paragraph_1.parquet'

# Parquet 파일 로드
train_df_check = pd.read_parquet(train_path)
test_df_check = pd.read_parquet(test_path)

# 데이터 확인
print("✅ train_paragraph_1.parquet shape:", train_df_check.shape)
print("✅ test_paragraph_1.parquet shape:", test_df_check.shape)
print("\n✅ train_df_check columns:")
print(train_df_check.columns.tolist()[:10])  # 일부만 출력

# 샘플 확인
print("\n✅ train_df_check sample:")
print(train_df_check.head(2))

train_npy_path = BASE_DIR + 'data/embeddings/train_concat_full.npy'
test_npy_path  = BASE_DIR + 'data/embeddings/test_concat_full.npy'

X_train_check = np.load(train_npy_path)
X_test_check  = np.load(test_npy_path)

# Shape 확인
print("✅ X_train shape:", X_train_check.shape)
print("✅ X_test shape:", X_test_check.shape)

# 값 확인
print("\n✅ X_train dtype:", X_train_check.dtype)
print("✅ X_train sample row 0 (first 5 features):", X_train_check[0][:5])

✅ train_paragraph_1.parquet shape: (1226364, 1549)
✅ test_paragraph_1.parquet shape: (1962, 1549)

✅ train_df_check columns:
['title', 'paragraph_index', 'paragraph_text', 'generated', 'char_count', 'word_count', 'sentence_count', 'absolute_position', 'relative_position', 'tfidf_mean']

✅ train_df_check sample:
    title  paragraph_index                                     paragraph_text  \
0  카호올라웨섬                0  카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...   
1  카호올라웨섬                1   마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에...   

   generated  char_count  word_count  sentence_count  absolute_position  \
0          0         112          22               3                  0   
1          0         169          40               3                  1   

   relative_position  tfidf_mean  ...  sbert_max_760  sbert_max_761  \
0               0.00    0.000306  ...      -0.549341       0.909734   
1               0.25    0.000437  ...      -0.549341       0.909734   

   